# Example of the `aitlas` toolbox in the context of object detection

## This notebook shows a sample implementation of object detection using the `aitlas` toolbox on the DOTAv1.0 (Task 2) dataset.

### Before using the object detection code provided in `aitlas` on the DOTAv1.0 dataset, it is important to split the images in the train/validation/test subsets into patches using the code available in this [Development kit](https://github.com/CAPTAIN-WHU/DOTA_devkit).

### The structure of the directories which hold the split images should be the following:
```
trainsplit/valsplit/testsplit
└───images
│   │   file011.png
│   │   file012.png
│   │   ...
│   
└───labelTxt
    │   file011.txt
    │   file012.txt
    │   ...
```

### A link to the DOTA project: [https://captain-whu.github.io/DOTA/dataset.html](https://captain-whu.github.io/DOTA/dataset.html)

In [ ]:
# datasets
from aitlas.datasets.esri_challenge import EsriChallengeDataset
from aitlas.datasets.dota import DotaDataset

# models
from aitlas.models.fastrcnn_detector import FastRCNN
from aitlas.models.retinanet import RetinaNet

# vizualization
from matplotlib import pyplot as plt
from matplotlib import patches
import matplotlib as mpl
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2

## Define configurations for all subsets of the dataset

### Configuration parameters:
```
- root (str): The path where the train/val/test splits can be found
- subset (str): The name of the folder inside the root directory which contains the images and labels of the split
- subsample_percentage (float): A percentage of the subset images which sould be used
- batch_size (int): Pytorch batch_size parameter
- shuffle (bool): Pytorch dataloader shuffle parameter
- num_workers (int): Pytorch dataloader num_workers parameter
- filter_null (bool): Whether to remove the images(patches) without any bounding boxes
- transforms (list): A list of transformation functions to be applied to the images in the subset
```

In [ ]:
data_path = "D:\\Documents\\AiTLAS\\aitlas\\ESRI-challenge\\data\\DOTA"

In [ ]:
train_cfg = {
    "root": data_path,
    "subset": "validationsplit",
    "subsample_percentage": 0.3,
    "batch_size": 2,
    "shuffle": False,
    "num_workers": 0,
    "filter_null": True,
    "transforms": ["torchvision.transforms.ToTensor"]
}
train_set = DotaDataset(train_cfg)

val_cfg = {
    "root": data_path,
    "subset": "validationsplit",
    "subsample_percentage": 0.3,
    "batch_size": 2,
    "shuffle": False,
    "num_workers": 0,
    "filter_null": True, 
    "transforms": ["torchvision.transforms.ToTensor"]
}
val_set = DotaDataset(val_cfg)

test_cfg = {
    "root": data_path,
    "subset": "testsplit",
    # this parameter does not matter when `test` is in the name of the subset
    "subsample_percentage": 1.0,
    "batch_size": 2,
    "shuffle": False,
    "num_workers": 0,
    # this parameter does not matter when `test` is in the name of the subset
    "filter_null": True, 
    "transforms": ["torchvision.transforms.ToTensor"]
}
test_set = DotaDataset(test_cfg)

## Plot 40 examples from the training subset along with their annotations

In [ ]:
num_examples = 40
num_images = train_set.__len__()

example_ind = np.random.randint(0, num_images, size = num_examples)

In [ ]:
fig, axes = plt.subplots(nrows = 5, ncols = 8, figsize = (22, 15))

for (example_idx, ax) in zip(example_ind, axes.flatten()):
    image, labels = train_set.__getitem__(example_idx)
    img = image.cpu().numpy().copy()
    img = np.rollaxis(img, 0, 3)
    img = (img*255).astype(np.uint16)

    ax.imshow(img)

    for (box, label) in zip(labels['boxes'].cpu().numpy().astype(np.int32), labels['labels'].cpu().numpy()):        
        tab20 = mpl.cm.get_cmap('tab20', 20)
        ax.add_patch(patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1], edgecolor=tab20(label), linewidth = 2, facecolor='none'))
        ax.add_patch(patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1], edgecolor=tab20(label), facecolor=tab20(label), alpha=0.3))
        ax.axis('off')

    
    tab20 = mpl.cm.get_cmap('tab20', 20)
    legend_items = []
    for cat, value in train_set.mappings.items():
        legend_items.append(patches.Patch(color=tab20(value), label=cat))
    plt.legend(loc = 'upper left', bbox_to_anchor=(1.05, 4.), handles = legend_items)
        
plt.show()

## Define RetinaNet as the model to use for detection

In [ ]:
# DOTA v1.0 has 15 unique classes + a background class
num_classes = 16

In [ ]:
model_cfg = {"num_classes": num_classes, "learning_rate": 0.001, "pretrained": True}

model = RetinaNet(model_cfg)

model.prepare()

## Train a RetinaNet model on the training subset and evaluate it after every epoch using the validation subset

In [ ]:
num_epochs = 20
log_interval = 100

In [ ]:
model.train_and_evaluate_model(
    train_dataset=train_set,
    val_dataset=val_set,
    epochs=num_epochs,
    model_directory = "./retinanet_dota/",
    run_id = "{}-retinanet".format("DOTA"),
    iterations_log = log_interval,
)

## Make predictions and save the predictions on disk

### The output directory must exist on disk

In [ ]:
val_output_path = "./task2_val_predictions"
test_output_path = "./task2_test_predictions"

In [ ]:
val_predictions = model.predict_with_output(val_set, description = 'Predicting the validation set')
val_set.add_predictions(val_predictions)
val_set.save_predictions(val_output_path)

In [ ]:
test_predictions = model.predict_with_output(test_set, description = 'Predicting the test set')
test_set.add_predictions(test_predictions)
test_set.save_predictions(test_output_path)